In [9]:
!pip install lxml
!pip install pytesseract Pillow
!pip install atlassian-python-api
!pip install langchain
!pip install tokenizers
!pip install redis
!pip install python-dotenv
!pip install tiktoken

  Using cached tiktoken-0.4.0-cp311-cp311-macosx_10_9_x86_64.whl (797 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.7/294.7 kB 1.1 MB/s eta 0:00:001.1 MB/s eta 0:00:01


In [27]:
from confluence import ConfluenceLoader, ContentFormat
import lxml
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key_str = os.environ['CONFLUENCE_API_KEY']
print(api_key_str)
token_str = os.environ['CONFLUENCE_TOKEN']
print(f"current token is {token_str}")

loader = ConfluenceLoader(
    url = "https://confluence.zhenguanyu.com",
    cloud = False,
    token = token_str
)
page_id = 192873905
raw_cql = f"type = page AND ancestor = {page_id}"
documents = loader.load(space_key="bigdataDoc", include_attachments=False, content_format=ContentFormat.VIEW)

PKNjrgMy4XvOJwD0VUapSU61qDf0oTfoX3Obnq
current token is ODM5ODU5NDkyNDM5OuUkqSppg/23bPIO2PKiLrEcYfio


In [28]:
print(len(documents))
print(documents[0])

249
lc_kwargs={'page_content': '一、产品介绍 Sonic 计算平台提供 低延迟、高吞吐、高准确性 的数据处理能力。 支持通过Flink、Spark等多种计算引擎创建 实时、离线 部署，可通过 SQL 语句、 JAR 包等方式，实现复杂业务逻辑数据处理，简单易用，为用户提供计算作业的全生命周期管理。 二、产品优势 Sonic计算平台有以下主要功能： 打通多个智能云存储数据，包括：Kafka，Hive，ES等 支持SQL、JAR包等多种方式任务开发 实时、离线数据查询功能 计算任务全面监控，全周期管理功能 部署版本管理，且可进行版本恢复 元数据管理功能 支持用户自定义函数 三、应用场景 核心指标实时/离线报表：多维度实时/离线监测PV/UV、销量、销售额、续报率等走势 个性化推荐：根据用户的访问内容，为用户推荐个性化内容 直播课实时监控：通过对日志文件进行实时分析，及时发现视频中顿率、延迟、丢包等问题 详情说明，请参考： 有任何的需求和意见可以联系 基础研发部-大数据平台 @gaoyangbj @zhangnanbj @wangpengbj', 'metadata': {'title': 'Sonic 计算平台', 'id': '81453268', 'source': 'https://confluence.zhenguanyu.com/pages/viewpage.action?pageId=81453268'}} page_content='一、产品介绍 Sonic 计算平台提供 低延迟、高吞吐、高准确性 的数据处理能力。 支持通过Flink、Spark等多种计算引擎创建 实时、离线 部署，可通过 SQL 语句、 JAR 包等方式，实现复杂业务逻辑数据处理，简单易用，为用户提供计算作业的全生命周期管理。 二、产品优势 Sonic计算平台有以下主要功能： 打通多个智能云存储数据，包括：Kafka，Hive，ES等 支持SQL、JAR包等多种方式任务开发 实时、离线数据查询功能 计算任务全面监控，全周期管理功能 部署版本管理，且可进行版本恢复 元数据管理功能 支持用户自定义函数 三、应用场景 核心指标实时/离线报表：多维度实时/离线监测PV/UV、销量、销售额、续报率等走势 个性化推荐：根据用户的访问内容，为用户推荐个性化内容 直播课实

In [29]:
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function = len)
docs = text_splitter.split_documents(documents)
print(len(docs))

681


In [30]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os

embeddings = OpenAIEmbeddings(deployment="embeddings_model", chunk_size=1)
db = Redis.from_documents(docs, embeddings, redis_url="redis://10.134.5.70:6379", index_name='chat_confluence_new')

In [24]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain, _get_chat_history
from typing import Dict, Any, Optional, List
from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.schema import Document

class CustomConversationalRetrievalChain(ConversationalRetrievalChain):
    last_source_documents: List[Document] = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.last_source_documents = None
        
    def _call(self, inputs: Dict[str, Any], run_manager: Optional[CallbackManagerForChainRun]=None) -> Dict[str, Any]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        question = inputs['question']
        get_chat_history = self.get_chat_history or _get_chat_history
        chat_history_str = get_chat_history(inputs['chat_history'])
        if chat_history_str:
            callbacks = _run_manager.get_child()
            new_question = self.question_generator.run(question=question, chat_history=chat_history_str, callbacks=callbacks)
        else:
            new_question = question
        docs = self._get_docs(new_question, inputs)
        new_inputs = inputs.copy()
        new_inputs['question'] = new_question
        new_inputs['chat_history'] = chat_history_str
        answer = self.combine_docs_chain.run(input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs)
        self.last_source_documents = docs
        return {self.output_key: answer}



In [26]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores.redis import Redis
from langchain.embeddings.openai import OpenAIEmbeddings
import langchain
import os
embeddings = OpenAIEmbeddings(deployment="embeddings_model", chunk_size=1)

query_db = Redis.from_existing_index(embeddings, redis_url="redis://10.134.5.70:6379", index_name='chat_confluence_new')
retriever = query_db.as_retriever()
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv, find_dotenv

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = AzureChatOpenAI(deployment_name="chat_confluence",model_name="gpt-35-turbo", verbose=True)
qa = CustomConversationalRetrievalChain.from_llm(llm, retriever, memory=memory)
langchain.debug=True
res1 = qa({"question": "How to create a service?"})
# print(f"answer is {res1}, source_docs is {qa.last_source_documents}")
res2 = qa({"question": ""})
# print(f"res2 is {res2}, source_docs are {qa.last_source_documents}")
langchain.debug=False

[chain/start] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "How to create a service?",
  "chat_history": []
}
[chain/start] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 2:RunTypeEnum.chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 2:RunTypeEnum.chain:StuffDocumentsChain > 3:RunTypeEnum.chain:LLMChain] Entering Chain run with input:
{
  "question": "How to create a service?",
  "context": "1 如何在 Console 上新建服务，需要提供什么信息 点击Console右上角的oncall，提oncall处理。oncall需要填写的信息如下： 系统：Console 类型：问题反馈类 概要：新建服务 gerrit项目名：你的项目名，如infra-fake-project 服务名：你的服务名，如：infra-fake-service 描述： 服务类型：http/rpc，静态服务，xxl-job服务，三选一 服务端口：如8080 如下图： 提交oncall后，会有专人处理（一般10min之内会处理）。 2 如何申请服务所需的机器资源 新建服务后，会在『服务列表』中展示对应的服务。 如果是service平台部署，点击『 申请service平台服务工单 』；申请工单后， 找运维审核 。 如果是k8s 部署，点『 修改k8s服务环境信息 』，填写好对应信息后，保存即可， 无需审核 （k8s在部署时，会自动调度机器资源）。 如下图：\n\n1 如何在 Console 上新建服务，需

[llm/end] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 2:RunTypeEnum.chain:LLMChain > 3:RunTypeEnum.llm:AzureChatOpenAI] [976.818ms] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "What is the process for creating a service and how long does it take for a specialist to handle it?",
        "generation_info": null,
        "message": {
          "content": "What is the process for creating a service and how long does it take for a specialist to handle it?",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 21,
      "prompt_tokens": 272,
      "total_tokens": 293
    },
    "model_name": "gpt-35-turbo"
  },
  "run": null
}
[chain/end] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 2:RunTypeEnum.chain:LLMChain] [978.202ms] Exiting Chain run with output:
{
  "text": "What is the process for creating a service and how long does 